In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget

In [4]:
%reload_ext autoreload
%autoreload 2

### Profile Likelihood

In [5]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [6]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [246]:
gfw.scale_Z2

1.8430945872854198e+17

In [8]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.69e+02
reduced chi2:  441.1817586533457
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  489.117997931911
***************************************************
Z2-amplitude:  2.74e+02
reduced chi2:  367.35685535243334
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  442.30175900202806
***************************************************
Z2-amplitude:  2.80e+02
reduced chi2:  462.42977088215497
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  302.2298893684773
***************************************************
Z2-amplitude:  2.69e+02
reduced chi2:  554.2181314149083
***************************************************
Z2-amplitude:  2.65e+02
reduced chi2:  657.0098909281799
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  512.0367862193627
*****************

### Signal model

In [9]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [10]:
test_yoffset = np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100))))

100%|██████████| 100/100 [00:02<00:00, 62.27it/s]


In [11]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(0,10000,100), test_yoffset)
print(np.mean(test_yoffset))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

13.9887662292


In [12]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 769.81it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


In [13]:
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force


### PL - average before fit

In [564]:
index = np.random.choice(10000, 10000, replace=False)

In [14]:
def func_filter(i):
#     ii = index[i]
    ii = i
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/chas_tests/77/Noise_batch_'+str(ii)+'.h5'
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/Noise_add_3_'+str(ii)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(ii)+'.h5'
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/nadav_test/0/Noise_batch_'+str(ii)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    
    res = bb.z2
    return (res)

In [367]:
del test_phases

In [15]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [01:50<00:00, 90.44it/s]


In [18]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [28]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_bdf3(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = np.array([1])
    noises_std =  func_sideband(data_z2, gravity_freqs)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
#     PL_array = A_array*0
#     if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
#         PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#         PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return mm_tmp.values[0]

In [89]:
gravity_freqs_all, calc_bdf3(100)

MLEs:  <ValueView of Minuit at 7fc19801da38>
  A: -4.03983372640505
  phi: 0.0
  sigma: 862.149185229033


(array([ 9, 12, 21, 24, 33, 39]), -4.03983372640505)

In [113]:
MLEs_all_half_freq4 = np.array(Parallel(n_jobs=40)(delayed(calc_bdf3)(i) for i in tqdm(range(10000))))






  0%|          | 0/10000 [00:00<?, ?it/s]




  0%|          | 40/10000 [00:02<08:23, 19.78it/s]




  1%|          | 80/10000 [00:03<07:47, 21.23it/s]




  1%|          | 120/10000 [00:04<06:03, 27.21it/s]




  2%|▏         | 160/10000 [00:04<04:43, 34.65it/s]




  2%|▏         | 200/10000 [00:04<03:39, 44.61it/s]




  2%|▏         | 240/10000 [00:05<02:56, 55.40it/s]




  3%|▎         | 280/10000 [00:05<02:26, 66.28it/s]




  3%|▎         | 320/10000 [00:05<02:07, 76.08it/s]




  4%|▎         | 360/10000 [00:06<01:52, 85.66it/s]




  4%|▍         | 400/10000 [00:06<01:44, 91.48it/s]




  4%|▍         | 440/10000 [00:06<01:38, 97.50it/s]




  5%|▍         | 480/10000 [00:07<01:31, 103.60it/s]




  5%|▌         | 520/10000 [00:07<01:25, 110.54it/s]




  6%|▌         | 560/10000 [00:07<01:23, 112.44it/s]




  6%|▌         | 600/10000 [00:08<01:23, 112.86it/s]




  6%|▋         | 640/10000 [00:08<01:18, 118.98it/s]




  7%|▋         | 680/10000 [00:08<01:16, 121.25it/

 90%|████████▉ | 8960/10000 [01:11<00:07, 132.28it/s]




 90%|█████████ | 9000/10000 [01:11<00:07, 129.97it/s]




 90%|█████████ | 9040/10000 [01:12<00:07, 131.06it/s]




 91%|█████████ | 9080/10000 [01:12<00:07, 131.12it/s]




 91%|█████████ | 9120/10000 [01:12<00:06, 130.56it/s]




 92%|█████████▏| 9160/10000 [01:12<00:06, 130.23it/s]




 92%|█████████▏| 9200/10000 [01:13<00:06, 126.53it/s]




 92%|█████████▏| 9240/10000 [01:13<00:05, 129.40it/s]




 93%|█████████▎| 9280/10000 [01:13<00:05, 130.18it/s]




 93%|█████████▎| 9320/10000 [01:14<00:05, 130.06it/s]




 94%|█████████▎| 9360/10000 [01:14<00:04, 131.42it/s]




 94%|█████████▍| 9400/10000 [01:14<00:04, 134.08it/s]




 94%|█████████▍| 9440/10000 [01:15<00:04, 130.80it/s]




 95%|█████████▍| 9480/10000 [01:15<00:04, 127.52it/s]




 95%|█████████▌| 9520/10000 [01:15<00:03, 132.19it/s]




 96%|█████████▌| 9560/10000 [01:16<00:03, 132.47it/s]




 96%|█████████▌| 9600/10000 [01:16<00:03, 130.35it/s]




 96%|█████████

In [19]:
gravity_freqs_all

array([ 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39])

In [55]:
MLEs = []

In [112]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,14)[2:]
# gravity_freqs_all = np.array([18, 21, 30, 33, 36, 39])
# gravity_freqs_all = np.array([9,12,21,24,33,39])
# gravity_freqs_all = np.array([15,18,27,30,36,42])
gravity_freqs_all = np.array([12,18,21,30,33,36,39])
gravity_freqs_all = np.array([12,36])
G_lambda = 10

test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])
A_array = np.arange(50,100,1)

fft_angles = np.angle(np.fft.rfft(shaking378_template))
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)
gravity_freqs = gravity_freqs_all
print(gravity_phases)
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
print(gravity_freqs_all, gravity_phases, gravity_scales)
# MLEs.append(Parallel(n_jobs=46)(delayed(calc_bdf3)(i) for i in tqdm(range(10000))))

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data
[array([ 2.06806183]), array([ 1.84123774])]
[12 36] [array([ 2.06806183]), array([ 1.84123774])] [  1.02044073e+17   1.40564888e+17]


In [70]:
len(MLEs)

11

In [71]:
MLEs = np.array(MLEs)
MLEs.shape

(11, 10000)

In [231]:
PLs= np.array(Parallel(n_jobs=40)(delayed(calc_bdf3)(i) for i in tqdm(range(2))))


100%|██████████| 2/2 [00:00<00:00,  3.76it/s]

In [202]:
PLs.shape

(2, 200)

In [153]:
import copy

PLs_conservative = []
PLscopy = copy.deepcopy(PLs)
for pll in PLscopy:
    PLs_tmp4 = []
    for pl in pll.copy():
        min_alpha = A_array[np.argmin(pl)]
        min_value = pl[np.argmin(pl)]
        if (min_alpha>-0.001):
            for i,A_ in enumerate(A_array):
                if A_<min_alpha:
                    pl[i] = min_value
            PLs_tmp4.append(pl)
        else:
            for i,A_ in enumerate(A_array):
                if A_>min_alpha:
                    pl[i] = min_value
            PLs_tmp4.append(pl)
#     PLs_tmp4 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_tmp4))
    PLs_conservative.append(PLs_tmp4)

TypeError: 'float' object is not subscriptable

In [232]:
PLs_conservative = []
PLscopy = copy.deepcopy(PLs)
for pl in PLscopy:
    min_alpha = A_array[np.argmin(pl)]
    min_value = pl[np.argmin(pl)]
    if (min_alpha>-0.001):
        for i,A_ in enumerate(A_array):
            if A_<min_alpha:
                pl[i] = min_value
        pl -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, pl))
        PLs_conservative.append(pl)
    else:
        for i,A_ in enumerate(A_array):
            if A_>min_alpha:
                pl[i] = min_value
        pl -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, pl))
        PLs_conservative.append(pl)
#     PLs_tmp4 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_tmp4))

In [62]:
PLs_all = []
PLscopy = copy.deepcopy(PLs)
for PLs_ in PLscopy:
    PLs5_tmp = PLs_[0]
    for pl in PLs_[1:]:
        PLs5_tmp += pl
    PLs5_tmp -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs5_tmp))
    PLs_all.append(PLs5_tmp)

In [233]:
_,ax = plt.subplots()
# _ = [ax.plot(A_array, pp) for pp in PLs]
_ = [ax.plot(A_array, pp) for pp in PLs_conservative]
_ = ax.set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$', ylabel = 'Profile-Likelihood')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [213]:
PLs_conservative = []
PLscopy = copy.deepcopy(PLs_all)
for pl in PLscopy:
    min_alpha = A_array[np.argmin(pl)]
    min_value = pl[np.argmin(pl)]
    if (min_alpha>(np.min(A_array)-0.2))&(min_alpha<(np.max(A_array)+0.2)):
        if (min_alpha>-0.001):
            for i,A_ in enumerate(A_array):
                if A_<min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)
        else:
            for i,A_ in enumerate(A_array):
                if A_>min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)

IndexError: index 200 is out of bounds for axis 0 with size 200

In [721]:
fig ,ax = plt.subplots(1,4,figsize=(9,2.5), sharey=True)
_ = [a.set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$') for a in ax[1:]]
_ = ax[0].set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
for i,harmonics in enumerate([2 ,10, 11, 12]):
    ax[i].plot(A_array, PLs_all[harmonics-1], label=str(harmonics*3)+' Hz.-shaking')
#     ax[i].plot(A_array, PLs_conservative[harmonics-1], label=str(harmonics*3)+' Hz.-shaking')
    ax[i].plot(A_array, A_array*0 + 2.71, 'r--')
_ = [a.legend() for a in ax]
fig.suptitle(r'PL vs. $\alpha_{fit} \, [10^{8}]$', fontsize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'PL vs. $\\alpha_{fit} \\, [10^{8}]$')

In [222]:
_,ax = plt.subplots()

PLs_summed_harmonics = []
PLscopy = copy.deepcopy(PLs_conservative)
PLs_summed_harmonics = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics += pp_

# PLs_summed_harmonics2 = []
# PLscopy = copy.deepcopy(PLs_all2)
# PLs_summed_harmonics2 = PLscopy[2]
# for pp_ in PLscopy[3:]:
#     PLs_summed_harmonics2 += pp_

PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics))
i = 0
ax.plot(A_array, PLs_summed_harmonics, '*-')
# ax.plot(A_array, PLs_all2[i], '*-')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{signal} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
# print(np.interp(2.71, PLs_summed_harmonics[100:], A_array[100:]))
# print(np.interp(2.71, PLs_summed_harmonics[100:0:-1], A_array[100:0:-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


[(0.0, 10.0),
 (-20.0, 19.900000000000567),
 Text(0.5, 0, '$\\alpha_{signal} \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood')]

### Limit

In [819]:

def getLim(g_lambda):
    print('**************************************************************')
    print('lambda: ', g_lambda)
    print('**************************************************************')
    ## calculate 100 likelihood finctions
    gravity_freqs_all = 3*np.arange(1,14)
    test_shaking378 = force_vs_time(dist,z_sep,stroke,3,g_lambda,"z", 
                                    yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
    shaking378_template = np.array(test_shaking378[1])
    A_array = np.arange(0,20,0.1)
    if (g_lambda<4.5):
        A_array = np.arange(-100,100,1)
    
    fft_angles = np.angle(np.fft.rfft(shaking378_template))
    freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
    fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
    tf_z = BeadDataFile.BeadDataFile(fname)
    fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
    freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
    fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
    freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

    PLs = []
    for drive_freq in gravity_freqs_all:
        phase_th = fft_angles[freq1==drive_freq]  ## theory 
        phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
        phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
        phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
        gravity_phases = np.array([phase_all])
        gravity_freqs = np.array([drive_freq])
        gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
        
        ll = likelihood_analyser.LikelihoodAnalyser()
        def calc_bdf_local(i):

            data_z2 = np.mean(test_phases[i*1000:(i+1)*1000], axis=0)

            fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
                      'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
                      'limit_phi': [0, np.pi],
                      'limit_A': [-10000, 10000], 'limit_sigma': [0, 1000000],
                      'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}

            noises_std = np.array([1])

            mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                                  signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                                  decimate=10, **fit_kwargs)
            print('MLEs: ', mm_tmp.values)
            PL_array = A_array*0
            if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
                PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        
#             PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#             PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))

            return PL_array
        PLs.append(Parallel(n_jobs=40)(delayed(calc_bdf_local)(i) for i in tqdm(range(1))))
        
    ## PL with the "new" construction
    
    ## option 1: sum all PLs for each harmonic and than make the construction
#     PLs_all = []
#     PLscopy = copy.deepcopy(PLs)
#     for PLs_ in PLscopy:
#         PLs5_tmp = PLs_[0]
#         for pl in PLs_[1:]:
#             PLs5_tmp += pl
#         PLs5_tmp -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs5_tmp))
#         PLs_all.append(PLs5_tmp)
        
#     PLs_conservative = []
#     PLscopy = copy.deepcopy(PLs_all)
#     for pl in PLscopy:
#         min_alpha = A_array[np.argmin(pl)]
#         min_value = pl[np.argmin(pl)]
#         if (min_alpha>(np.min(A_array)-0.2))&(min_alpha<(np.max(A_array)+0.2)):
#             if (min_alpha>-0.001):
#                 for i,A_ in enumerate(A_array):
#                     if A_<min_alpha:
#                         pl[i] = min_value
#                 PLs_conservative.append(pl)
#             else:
#                 for i,A_ in enumerate(A_array):
#                     if A_>min_alpha:
#                         pl[i] = min_value
#                 PLs_conservative.append(pl)
#     del PLscopy

#     ## sum up all harmonics
#     PLs_summed_harmonics = []
#     PLscopy = copy.deepcopy(PLs_conservative)
#     PLs_summed_harmonics = PLscopy[0]
#     for pp_ in PLscopy[1:]:
#         PLs_summed_harmonics += pp_
        
    ## end of option 1
    
    ## option 2: make the construction and than some all PLs for each harmonic
    PLs_conservative = []
    PLscopy = copy.deepcopy(PLs)
    for pll in PLscopy:
        PLs_tmp4 = []
        for pl in pll.copy():
            min_alpha = A_array[np.argmin(pl)]
            min_value = pl[np.argmin(pl)]
            if (min_alpha>-0.001):
                for i,A_ in enumerate(A_array):
                    if A_<min_alpha:
                        pl[i] = min_value
                PLs_tmp4.append(pl)
            else:
                for i,A_ in enumerate(A_array):
                    if A_>min_alpha:
                        pl[i] = min_value
                PLs_tmp4.append(pl)
        PLs_conservative.append(PLs_tmp4)
    del PLscopy
    
    PLs_all = []
    PLscopy = copy.deepcopy(PLs_conservative)
    for PLs_ in PLscopy:
        PLs5_tmp = PLs_[0]
        for pl in PLs_[1:]:
            PLs5_tmp += pl
        PLs5_tmp -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs5_tmp))
        PLs_all.append(PLs5_tmp)
    del PLscopy
    
    ## sum up all harmonics
    PLs_summed_harmonics = []
    PLscopy = copy.deepcopy(PLs_all)
    PLs_summed_harmonics = PLscopy[0]
    for pp_ in PLscopy[1:]:
        PLs_summed_harmonics += pp_
    ## end of option 2

    PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics))
    pos_lim = np.interp(2.71, PLs_summed_harmonics[np.argmin(PLs_summed_harmonics):], A_array[np.argmin(PLs_summed_harmonics):])
#     neg_lim = np.interp(2.71, PLs_summed_harmonics[np.argmin(PLs_summed_harmonics):0:-1], A_array[np.argmin(PLs_summed_harmonics):0:-1])
    return PLs_summed_harmonics

In [820]:
PLtmp2 = getLim(9.74)

**************************************************************
lambda:  9.74
**************************************************************
Loading Gravity Data... 








  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 946.80it/s]

Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data









  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 593.76it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 452.46it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 663.13it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 639.38it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 667.67it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 612.49it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 620.28it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 633.48it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 590.00it/s]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00, 618.81it/s]






  0%|          | 0/1 [0

In [821]:
A_array = np.arange(0,20,0.1)
_,ax = plt.subplots()
ax.plot(A_array, PLtmp2, '*--')
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{signal} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
np.interp(2.71, PLtmp[np.argmin(PLtmp):], A_array[np.argmin(PLtmp):])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

10.731039864689425

In [656]:
L = np.logspace(0.4, 1.5, 20)
alpha_pos_lim3 = []
# alpha_neg_lim = []
for i,L_ in enumerate(L):
    print(i,'/10')
    alpha_p, alpha_n = getLim(L_)
    alpha_pos_lim3.append(alpha_p)
#     alpha_neg_lim.append(alpha_n)

0 /10
**************************************************************
lambda:  2.51188643151
**************************************************************
Loading Gravity Data... 






  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5701.11it/s]

Done!
105 2.46954787382e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data







  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5885.92it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5401.55it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6262.96it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 7453.89it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 7590.13it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 7496.52it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5974.79it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6295.86it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5624.65it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 10551.71it/s]




  0%|          | 0/10 [0

1 /10
**************************************************************
lambda:  2.87008485407
**************************************************************
Loading Gravity Data... Done!
102 2.93166332444e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


...






  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 2261.69it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 718.79it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 3933.51it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 1762.46it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6687.35it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5203.85it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 2714.06it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6145.50it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 4017.53it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 2419.98it/s]




  0%|          | 0/10 [00:

19 /10
**************************************************************
lambda:  31.6227766017
**************************************************************
Loading Gravity Data... Done!
60 3.23643667635e-05
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data







  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6170.82it/s]

No laser power data







  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 5925.00it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 1044.45it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 1835.26it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6367.55it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 3578.15it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 6794.60it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 1308.76it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 2117.37it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 4050.12it/s]




  0%|          | 0/10 [00:00<?, ?it/s]




100%|██████████| 10/10 [00:00<00:00, 10338.44it/s]




  0%|          | 0/10 [0

In [657]:
L1 = np.logspace(0.4, 1.5, 20)
L2 = np.logspace(0.4, 1.5, 10)

_, ax = plt.subplots(figsize=(7,5))
ax.set(xlim=(1,1e2), ylim=(1e8,5e10), xlabel='$\lambda [\mu m]$', ylabel=r'$\alpha$')
ax.loglog(L2, np.abs(alpha_pos_lim4)*1e8,'--', label=r'add_3')
ax.loglog(L2, np.abs(alpha_pos_lim5)*1e8,'--', label=r'chas-77-Noise_batch')
ax.loglog(L1, np.abs(alpha_pos_lim3)*1e8,'--', label=r'nadav-0-Noise_batch')
# ax.loglog(L1, np.abs(alpha_pos_lim)*1e8,'--', label=r'Wilson positive')
# ax.loglog(L1, np.abs(alpha_neg_lim)*1e8,'--', label=r'Wilson negative')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Signal injected files - MLEs

In [14]:
3*np.arange(2,14)

array([ 6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39])

In [126]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(2,14)
G_lambda = 10

test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

fft_angles = np.angle(np.fft.rfft(shaking378_template))
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

PLs = []
gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)
gravity_freqs = gravity_freqs_all
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])

all_mles_SBiN2_10 = []
for j in range(15):
    print('**************************************************************')
    print('j: ', j)
    print('**************************************************************')
    def func_filter(i):
        ii = i
#         fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/Batch3/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'
#         fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/non_blind_with_bkg/Test9/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'
        fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/SBiN_2/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'

        bb = BeadDataFile.BeadDataFile(fname=fname)
        res = bb.z2
        return (res)
    
    test_phases = np.array(Parallel(n_jobs=10)(delayed(func_filter)(i) for i in tqdm(range(500))))
    
    ll = likelihood_analyser.LikelihoodAnalyser()

    def calc_mle(i):

        data_z2 = np.mean(test_phases[i*500:(i+1)*500], axis=0)

        fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
                  'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
                  'limit_phi': [0, np.pi],
                  'limit_A': [-10000, 10000], 'limit_sigma': [0, 1000000],
                  'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}

        noises_std =  func_sideband(data_z2, gravity_freqs)
        noises_std = noises_std/np.min(noises_std)

        mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                              signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                              decimate=10, **fit_kwargs)
        return mm_tmp.values[0]
    all_mles_SBiN2_10.append(Parallel(n_jobs=40)(delayed(calc_mle)(i) for i in tqdm(range(1))))

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force




  0%|          | 0/500 [00:00<?, ?it/s]

No height data
No cantilever data
Error loading spinning data
No laser power data
**************************************************************
j:  0
**************************************************************


Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = 

**************************************************************
j:  1
**************************************************************


...



  0%|          | 1/500 [00:00<02:50,  2.93it/s]

  2%|▏         | 10/500 [00:00<02:06,  3.88it/s]

  4%|▍         | 20/500 [00:01<01:37,  4.92it/s]

  6%|▌         | 30/500 [00:01<01:08,  6.88it/s]

 10%|█         | 50/500 [00:01<00:47,  9.57it/s]

 16%|█▌        | 80/500 [00:02<00:31, 13.32it/s]

 20%|██        | 100/500 [00:02<00:21, 18.47it/s]

 28%|██▊       | 140/500 [00:02<00:14, 25.34it/s]

 36%|███▌      | 180/500 [00:02<00:09, 33.75it/s]

 44%|████▍     | 220/500 [00:03<00:06, 42.28it/s]

 52%|█████▏    | 260/500 [00:03<00:04, 50.20it/s]

 60%|██████    | 300/500 [00:03<00:03, 60.81it/s]

 68%|██████▊   | 340/500 [00:04<00:02, 67.83it/s]

 76%|███████▌  | 380/500 [00:04<00:01, 78.19it/s]

 84%|████████▍ | 420/500 [00:05<00:00, 85.15it/s]

 92%|█████████▏| 460/500 [00:05<00:00, 91.22it/s]

100%|██████████| 500/500 [00:05<00:00, 98.40it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.53it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

**************************************************************
j:  13
**************************************************************




  0%|          | 1/500 [00:00<03:08,  2.65it/s]

  2%|▏         | 10/500 [00:00<02:17,  3.56it/s]

  4%|▍         | 20/500 [00:01<01:44,  4.61it/s]

  6%|▌         | 30/500 [00:01<01:13,  6.37it/s]

 10%|█         | 50/500 [00:01<00:50,  8.90it/s]

 16%|█▌        | 80/500 [00:02<00:33, 12.40it/s]

 20%|██        | 100/500 [00:02<00:23, 17.22it/s]

 28%|██▊       | 140/500 [00:02<00:15, 23.88it/s]

 36%|███▌      | 180/500 [00:02<00:10, 31.78it/s]

 44%|████▍     | 220/500 [00:03<00:06, 40.47it/s]

 52%|█████▏    | 260/500 [00:03<00:04, 49.44it/s]

 60%|██████    | 300/500 [00:03<00:03, 58.45it/s]

 68%|██████▊   | 340/500 [00:04<00:02, 67.51it/s]

 76%|███████▌  | 380/500 [00:04<00:01, 76.19it/s]

 84%|████████▍ | 420/500 [00:04<00:00, 82.76it/s]

 92%|█████████▏| 460/500 [00:05<00:00, 90.19it/s]

100%|██████████| 500/500 [00:05<00:00, 91.86it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.74it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

**************************************************************
j:  14
**************************************************************




  0%|          | 1/500 [00:00<03:04,  2.70it/s]

  2%|▏         | 10/500 [00:00<02:15,  3.61it/s]

  4%|▍         | 20/500 [00:01<01:43,  4.63it/s]

  6%|▌         | 30/500 [00:01<01:12,  6.47it/s]

  8%|▊         | 40/500 [00:01<00:51,  8.99it/s]

 12%|█▏        | 60/500 [00:02<00:35, 12.42it/s]

 16%|█▌        | 80/500 [00:02<00:24, 17.22it/s]

 20%|██        | 100/500 [00:02<00:17, 23.24it/s]

 24%|██▍       | 120/500 [00:02<00:12, 31.17it/s]

 28%|██▊       | 140/500 [00:02<00:09, 38.79it/s]

 32%|███▏      | 160/500 [00:03<00:10, 33.50it/s]

 36%|███▌      | 180/500 [00:03<00:07, 41.05it/s]

 40%|████      | 200/500 [00:03<00:05, 51.38it/s]

 44%|████▍     | 220/500 [00:04<00:04, 59.55it/s]

 48%|████▊     | 240/500 [00:04<00:03, 68.42it/s]

 52%|█████▏    | 260/500 [00:04<00:03, 78.75it/s]

 56%|█████▌    | 280/500 [00:04<00:02, 83.05it/s]

 60%|██████    | 300/500 [00:04<00:02, 87.36it/s]

 64%|██████▍   | 320/500 [00:05<00:01, 91.66it/s]

 68%|██████▊   | 340/500 [00:05<00:01

In [28]:
import copy

In [127]:
all_mles_SBiN2_10

[[3.490944809977009],
 [4.006355932515362],
 [1.3579884988957929],
 [-1.1709497155570716],
 [-0.6788132157471409],
 [5.004571420786306],
 [0.7361278974258312],
 [1.3452652784053498],
 [0.12576248084951658],
 [2.057554048848033],
 [3.7922371567856317],
 [1.751805534979212],
 [-0.38376006008729746],
 [4.395249464270819],
 [3.2761725348336768]]

In [243]:
dist,z_sep,stroke,3,G_lambda,y_offset

(13.988766229192864,
 -15.85910039296383,
 -202.76574813093302,
 3,
 10,
 4.882143696792518)

In [236]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,14)
G_lambda = 10
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

fft_angles = np.angle(np.fft.rfft(shaking378_template))
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

PLs = []
gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)
gravity_freqs = gravity_freqs_all
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])

_,ax = plt.subplots()
pos10 = []
neg10 = []
for j in range(1):
    print('**************************************************************')
    print('j: ', j)
    print('**************************************************************')
    def func_filter(i):
        ii = i
#         fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/Batch3/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'
#         fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/non_blind_with_bkg/Test9/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'
#         fname = r'/data/new_trap_processed/mockfiles/20200320/raw/noise/NoShaking_1_'+str(ii)+'.h5'
#         fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/SBiN_2/'+str(j)+'/Noise_batch_'+str(ii)+'.h5'
        fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/bkg_simple/0/0/Noise_batch_'+str(ii)+'.h5'
        bb = BeadDataFile.BeadDataFile(fname=fname)
        res = bb.z2
        return (res)
    
    test_phases = np.array(Parallel(n_jobs=10)(delayed(func_filter)(i) for i in tqdm(range(1000))))
    
    ll = likelihood_analyser.LikelihoodAnalyser()
    
    A_array = np.arange(-10,10,0.1)
    if (np.abs(all_mles_SBiN2_10[j])>10):
        A_array = np.arange(-100,100,1)
        
    def calc_lim(i):

        data_z2 = np.mean(test_phases[i*500:(i+1)*500], axis=0)

        fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
                  'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
                  'limit_phi': [0, np.pi],
                  'limit_A': [-10000, 10000], 'limit_sigma': [0, 1000000],
                  'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}

        noises_std =  func_sideband(data_z2, gravity_freqs)
        noises_std = noises_std/np.min(noises_std)

        mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                              signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                              decimate=10, **fit_kwargs)
        PL_array = A_array*0
        if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
            PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        
        return PL_array
    
    PLs = np.array(Parallel(n_jobs=40)(delayed(calc_lim)(i) for i in tqdm(range(2))))
    PLs_conservative = []
    PLscopy = copy.deepcopy(PLs)
    for pl in PLscopy:
        min_alpha = A_array[np.argmin(pl)]
        min_value = pl[np.argmin(pl)]
        if (min_alpha>-0.001):
            for i,A_ in enumerate(A_array):
                if A_<min_alpha:
                    pl[i] = min_value
            pl -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, pl))
            PLs_conservative.append(pl)
        else:
            for i,A_ in enumerate(A_array):
                if A_>min_alpha:
                    pl[i] = min_value
            pl -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, pl))
            PLs_conservative.append(pl)
    PLs_summed_harmonics = []
    PLscopy = copy.deepcopy(PLs_conservative)
    PLs_summed_harmonics = PLscopy[0]
    for pp_ in PLscopy[1:]:
        PLs_summed_harmonics += pp_
    
#     _,ax = plt.subplots()
#     ax.plot(A_array, PLs_summed_harmonics)
    tmp_n = int(len(PLs_summed_harmonics)/2)
    pos_lim = np.interp(2.71, PLs_summed_harmonics[tmp_n:], A_array[tmp_n:])
    neg_lim = np.interp(2.71, PLs_summed_harmonics[tmp_n:0:-1], A_array[tmp_n:0:-1])
    if(pos_lim<A_array[-2]):
        pos10.append(pos_lim)
    else:
        pos10.append(float("nan"))
    if(neg_lim>A_array[1]):
        neg10.append(neg_lim)
    else:
        neg10.append(float("nan"))
    print(pos10)
    print(neg10)
#     ax.plot(range(len(pos5)), pos5, '--')

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …



  0%|          | 0/1000 [00:00<?, ?it/s]

**************************************************************
j:  0
**************************************************************




  1%|          | 10/1000 [00:00<00:58, 16.96it/s]

  2%|▏         | 20/1000 [00:01<01:00, 16.14it/s]

  4%|▍         | 40/1000 [00:01<00:43, 22.16it/s]

  9%|▉         | 90/1000 [00:01<00:29, 30.95it/s]

 17%|█▋        | 170/1000 [00:01<00:19, 43.48it/s]

 33%|███▎      | 330/1000 [00:01<00:10, 61.00it/s]

 49%|████▉     | 490/1000 [00:01<00:05, 85.56it/s]

 65%|██████▌   | 650/1000 [00:02<00:02, 117.04it/s]

 81%|████████  | 810/1000 [00:02<00:01, 158.61it/s]

 97%|█████████▋| 970/1000 [00:02<00:00, 208.54it/s]

100%|██████████| 1000/1000 [00:02<00:00, 406.19it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:00<00:00,  3.51it/s]

100%|██████████| 2/2 [00:00<00:00,  6.96it/s]

[5.7157530733264563]
[nan]


In [242]:
_,ax = plt.subplots()
ax.plot(A_array, PLs[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(15), pos10)
ax.scatter(range(15), all_mles_SBiN2_10, label='all hramonics')
ax.scatter(range(15), neg10)
ax.legend()
ax.set(xlabel='dataset', ylabel=r'$\hat{\alpha} \,[10^{8}]$')

In [301]:
_,ax = plt.subplots()
ax.plot(range(100), pos5, '--')
ax.scatter(range(100), all_mles5)
ax.plot(range(100), neg5, '--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
pos_all_lim = np.array([pos5, pos10, pos25])
neg_all_lim = np.array([neg5, neg10, neg25])

In [123]:
mle_lim = np.array([neg10, all_mles_SBiN1_10, pos10])
np.savetxt('SBiN1_mle_NP.csv',all_mles_SBiN1_10,delimiter=',')

In [124]:
np.savetxt('SBiN1_positive_NP.csv',pos10,delimiter=',')

In [75]:
np.savetxt('Test5_positive_NP.csv',pos_all_lim,delimiter=',')

In [76]:
np.savetxt('Test5_negative_NP.csv',neg_all_lim,delimiter=',')

In [100]:
np.savetxt('Test9_mle_NP.csv',all_mles_test9_10,delimiter=',')

### MLEs

In [98]:
_,ax = plt.subplots()
ax.scatter(np.arange(3,3+len(MLEs))*3, np.std(MLEs, axis=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
np.mean(MLEs[:100],axis=1)

array([ 10.46150202,  -0.74199207,  -9.37579304,   1.36492677,
         7.74738345,  -5.0770016 ,   9.96721762,  -3.32933619,
         3.76784408,   1.44281519,   5.56992944])

In [156]:
MLEs.shape

(11, 10000)

In [165]:
limit_new = []
limit_new2 = []
for i in range(int(10000/downsample)):
    tmp = np.mean(MLEs.T[i*downsample:(i+1)*downsample],axis=1)
    limit_new.append(np.min(tmp[tmp>0]))
    limit_new2.append(np.max(tmp[tmp<0]))

In [180]:
downsample = 1000
data = MLEs_all

alphas_mean = [np.mean(data[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))]
alphas_std = [np.std(data[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))]

j1 = 3
data2 = MLEs[j1]
alphas_mean2 = [np.mean(data2[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))]
alphas_std2 = [np.std(data2[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))]

j2 = 9
data3 = MLEs
alphas_mean3 = np.array([np.mean(data3[i*downsample:(i+1)*downsample], axis=1) for i in range(int(10000/downsample))])
alphas_std3 = [np.std(data3[i*downsample:(i+1)*downsample], axis=1)/np.sqrt(downsample) for i in range(int(10000/downsample))]

limit_new = []
limit_new2 = []
for i in range(int(10000/downsample)):
    tmp = np.mean(MLEs.T[i*downsample:(i+1)*downsample],axis=0)
    limit_new.append(np.min(tmp[tmp>0]))
    limit_new2.append(np.max(tmp[tmp<0]))
#     print('pos: ', np.argmin(tmp[tmp>0]))
#     print('neg: ', np.argmax(tmp[tmp<0]))

_,ax = plt.subplots(figsize=(9.2,4))
rr_tmp = 15
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_mean), yerr=alphas_std, fmt='o', zorder=0, label='Multi-harmonic fit ')
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_mean2), yerr=alphas_std2, fmt='o', zorder=0, label=str(j1+3)+'th-harmonic fit ')
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(limit_new), yerr=alphas_std2, fmt='o', zorder=0, label='lim-pos')
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(limit_new2), yerr=alphas_std2, fmt='o', zorder=0, label='lim-neg')
ax.plot(np.arange(-100,10000,100)*10/60, np.arange(-100,10000,100)*0, 'r--', zorder=10)
ax.set(xlabel='Time [minute]', ylabel=r'$\alpha \,[10^{8}]$', ylim=(-rr_tmp,rr_tmp))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
_,ax = plt.subplots(figsize=(9.2,4))
rr_tmp = 15
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_std), yerr=np.array(alphas_std2)/100+np.std(alphas_std)*0, fmt='o', zorder=0, label='Multi-harmonic fit ')
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_std2), yerr=np.array(alphas_std2)/100+np.std(alphas_std2)*0, fmt='o', zorder=0, label=str(j1+3)+'th-harmonic fit ')
ax.errorbar(np.arange(0,10000,downsample)*10/60, np.array(alphas_std3), yerr=np.array(alphas_std2)/100+np.std(alphas_std3)*0, fmt='o', zorder=0, label=str(j2+3)+'th-harmonic fit ')
ax.set(xlabel='Time [minute]', ylabel=r'$\alpha \,[10^{8}]$', ylim=(0,10))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …